In [1]:
import streamlit as st
import pandas as pd
import folium
from branca.colormap import linear

import plotly.express as px

from plotly.subplots import make_subplots


In [2]:
df = pd.read_excel('precos_carrefour_empanado_20250516_tratado.xlsx')
df

,Unnamed: 0.2,Unnamed: 0.1,data,Unnamed: 0,produto,preco,desconto,loja,cidade,uf,lat,long,regiao
0,0,0,17/05/2025,0,Empanado Chicken Crispy Tradicional Seara 300g,12.49,NaN,Hiper Maceió JatiúcaMangabeiras,Maceió - AL,AL,-9.64959,-35.71739,Nordeste
1,1,1,17/05/2025,1,Iscas De Frango Tradicional Empanado Crocante ...,14.65,NaN,Hiper Maceió JatiúcaMangabeiras,Maceió - AL,AL,-9.64959,-35.71739,Nordeste
2,2,2,17/05/2025,2,Empanado de Frango Steak Recheio Presunto de P...,4.09,NaN,Hiper Maceió JatiúcaMangabeiras,Maceió - AL,AL,-9.64959,-35.71739,Nordeste
3,3,3,17/05/2025,3,Empanado De Frango Incrível 100% Vegetal 300g,17.85,-43%,Hiper Maceió JatiúcaMangabeiras,Maceió - AL,AL,-9.64959,-35.71739,Nordeste
4,4,4,17/05/2025,4,Empanado de Frango Vegetal Congelado Incrível!...,28.39,NaN,Hiper Maceió JatiúcaMangabeiras,Maceió - AL,AL,-9.64959,-35.71739,Nordeste
...,...,...,...,...,...,...,...,...,...,...,...,...,...
459,459,459,17/05/2025,459,Empanado de Frango Tradicional Crocante Swift ...,10.90,NaN,Carrefour Bairro MorumbiJardim Londrina,São Paulo - SP,SP,-23.61172,-46.74075,Sudeste
460,460,460,17/05/2025,460,Empanado de Frango com Queijo Crocante Swift 3...,10.90,NaN,Carrefour Bairro MorumbiJardim Londrina,São Paulo - SP,SP,-23.61172,-46.74075,Sudeste
461,461,461,17/05/2025,461,Atum Sólido em Óleo Gomes da Costa 170g,12.99,NaN,Carrefour Bairro MorumbiJardim Londrina,São Paulo - SP,SP,-23.61172,-46.74075,Sudeste
462,462,462,17/05/2025,462,Atum Sólido ao Natural Gomes da Costa 170g,12.99,NaN,Carrefour Bairro MorumbiJardim Londrina,São Paulo - SP,SP,-23.61172,-46.74075,Sudeste


In [3]:
num_cidades = df['cidade'].nunique()
print(f'Número de cidades pesquisadas: {num_cidades}')

Número de cidades pesquisadas: 61


In [4]:
num_produtos = df['produto'].nunique()
print(f'Número de produtos diferentes: {num_produtos}')

Número de produtos diferentes: 50


In [6]:
num_produtos_frango_empanado = df['produto'].str.lower().str.contains('peixe') & df['produto'].str.lower().str.contains('empanado')
num_produtos_frango_empanado = df.loc[num_produtos_frango_empanado, 'produto'].nunique()
print(f'Número de produtos com "frango" e "empanado" no nome: {num_produtos_frango_empanado}')

Número de produtos com "frango" e "empanado" no nome: 3


In [4]:
# Gráfico de barras do preço médio por cidade
df_cidade = df.groupby('cidade')['preco'].mean().reset_index()

fig = px.bar(df_cidade, x='cidade', y='preco', title='Preço médio por cidade', labels={'preco': 'Preço médio (R$)'})
fig.update_layout(xaxis_tickangle=-45)
fig.show()

In [3]:
df_produto = df.groupby('produto').agg(
    preco_medio=('preco', 'mean'),
    preco_min=('preco', 'min'),
    preco_max=('preco', 'max'),
    lojas=('loja', 'nunique'),
    cidades=('cidade', 'nunique')
).reset_index()

df_produto

,produto,preco_medio,preco_min,preco_max,lojas,cidades
0,Hossomaki Philadelphia Kani Kama 160 g,25.040000,22.59,27.49,2,2
1,Kani Congelado Costa Sul 200g,12.599091,9.89,16.49,43,43
2,Kani Congelado Damm 250g,14.890000,10.79,22.69,3,3
3,Kani Congelado Frescatto 200g,12.792927,8.99,15.99,40,40


In [ ]:
#

In [8]:
df_1 = df.loc[df['produto'] == 'Kani Congelado Frescatto 200g']

In [9]:
# Criar um mapa Folium centrado no Brasil
mapa = folium.Map(location=[-15.793889, -47.882778], zoom_start=4)

# Definir a escala de cores para os preços
preco_minimo = df_1['preco'].min()
preco_maximo = df_1['preco'].max()
colormap = linear.YlOrRd_09.scale(preco_minimo, preco_maximo)

# Adicionar círculos para representar os preços das lojas
for _, row in df_1.iterrows():
    circle = folium.CircleMarker(location=[row['lat'], row['long']], radius=10, color='black', fill=True, 
                                fill_opacity=0.7, fill_color=colormap(row['preco']))
    
    # Adicionar popup com o nome da loja e preço ao clicar no círculo
    popup_html = f"<b>{row['loja']}</b><br>Preço: <b>R${row['preco']:.2f}</b>"
    popup = folium.Popup(popup_html, parse_html=False)
    circle.add_child(popup)
    

    circle.add_to(mapa)

# Adicionar a legenda da escala de cores
colormap.caption = 'Preço'
colormap.add_to(mapa)

mapa 
# Exibir o mapa no Streamlit
#mapa.save("mapa.html")
#st.components.v1.html(open("mapa.html", "r").read(), height=600)